################  IMPORTING THE REQUIRED LIBRARIES

In [14]:
import numpy as np
import soundfile
import os, glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import pandas as pd
import librosa
from sklearn import preprocessing 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

##############   EMOTIONS INCLUDED IN THE DATASET

In [15]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

##############    EXTRACTING FEATURES FROM THE AUDIO SIGNAL USING LIBROSA

In [16]:
def extract_feature(file_name, mfcc, chroma,spectral_centroid,spectral_bandwidth,
                    spectral_rolloff,spectral_contrast,rms,spectral_flatness):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
            
            
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))             
           
            
        if spectral_centroid:
            spectral_centroid=np.mean(librosa.feature.spectral_centroid(y=X, sr=sample_rate))
            result=np.hstack((result, spectral_centroid)) 
        
        if spectral_bandwidth:
           spectral_bandwidth=np.mean(librosa.feature.spectral_bandwidth(y=X, sr=sample_rate).T)
#           print(spectral_bandwidth)
           result=np.hstack((result, spectral_bandwidth)) 
           
        if spectral_rolloff:
           spectral_rolloff=np.mean(librosa.feature.spectral_rolloff(y=X, sr=sample_rate).T)
#           print(spectral_rolloff)
           result=np.hstack((result, spectral_rolloff))
        
        if spectral_contrast:
           spectral_contrast=np.mean(librosa.feature.spectral_contrast(y=X, sr=sample_rate))
           result=np.hstack((result, spectral_contrast))
           
        if rms:
           rms=np.mean(librosa.feature.rms(y=X).T,axis=0)
           result=np.hstack((result, rms))
           
        if spectral_flatness:
           spectral_flatness=np.mean(librosa.feature.spectral_flatness(y=X))
           result=np.hstack((result, spectral_flatness))
        
        return result

##############     LOADING THE DATASET AND  EXTRACTING ALL THE FEATURES FROM IT    

In [17]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(r"F:\speech_project\speech-emotion-recognition-ravdess-data\Actor_*\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        feature=extract_feature(file, mfcc=True, chroma=True,spectral_centroid=True,spectral_bandwidth=True,
                                spectral_rolloff=True,spectral_contrast=True,rms=True,spectral_flatness=True)
        x.append(feature)
        y.append(emotion)
    return x,y


##############     SHUFFLING OF OBS AND RESETTING THE OBS INDEXES

In [18]:
X,y=load_data()
X=pd.DataFrame(X)
y=pd.DataFrame(y)
data=pd.concat([X,y],axis="columns")
data=data.sample(frac=1).reset_index(drop=True)
X=data.iloc[:,0:58]
y=data.iloc[:,58]

##############     LABEL ENCODING THE RESPONSE VARIABLES

In [19]:
le = preprocessing.LabelEncoder()
y=le.fit_transform(y)

##############     SPLITTING THE DATA INTO TRAIN(95%) AND TEST(5%)

In [20]:
x_train,x_test,y_train,y_test=train_test_split(X, y, test_size=0.05, random_state=42)
print((x_train.shape[0], x_test.shape[0]))

print(f'Features extracted: {x_train.shape[1]}')

(1345, 71)
Features extracted: 58


In [21]:
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)

###################  HYPER-PARAMETER TUNING

In [22]:
lr_range = [0.01, 0.1, 0.2,0.3,0.5]
n_est_range = [90,50,100,200]
md_range = [2,4,6,8,10]

parameters = {'learning_rate':lr_range,
                  'n_estimators':n_est_range,
                  'max_depth':md_range}


#################### TRAINING THE MODEL

In [33]:
clf = XGBClassifier(random_state=2019)
cv = GridSearchCV(clf, param_grid=parameters,cv=5,scoring='accuracy',verbose=3, n_jobs=-1)
cv.fit(x_train,y_train)
y_pred_xgboost=cv.predict(x_test)
print("Accuracy=",accuracy_score(y_test, y_pred_xgboost))
print(cv.best_params_)
print(cv.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  6.9min finished


Accuracy= 0.6197183098591549
{'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 90}
0.6304832713754646


############# SAVING THE MODEL

In [34]:
import pickle
pickle.dump(cv, open("C:\\Users\\dbda\\Desktop\\project\\XGB.pkl", 'wb'))
print("Model Saved!!")

Model Saved!!


#################### PREDICTION ON UNSEEN DATA

In [9]:

def load_test_data():
    x_test,y_test=[],[]
    
    file="F:\\speech_project\\UnSeenSet\\03-01-07-01-01-02-15.wav"
    emotion=emotions[file.split("-")[2]]
    feature=extract_feature(file, mfcc=True, chroma=True,spectral_centroid=True,spectral_bandwidth=True,
                            spectral_rolloff=True,spectral_contrast=True,rms=True,spectral_flatness=True)
    
    x_test.append(feature)
    y_test.append(emotion)
    print(x_test)
    return x_test,y_test

In [12]:
X_test_data,y_test_data=load_test_data()
#X_test_data=pd.DataFrame(X_test_data)
print(y_test_data)

NameError: name 'emotions' is not defined

########### LOADING SAVED MODEL AND PREDICTING

In [13]:
import pickle
loaded_model = pickle.load(open("C:\\Users\\dbda\\Desktop\\project\\XGB.pkl", 'rb'))
result = loaded_model.predict(X_test_data)
print(result)

NameError: name 'X_test_data' is not defined

In [ ]:
print(le.inverse_transform(result))